In [9]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_regression as mi_reg
from ff_custom_scripts import *


In [10]:
background = pd.read_csv('../data/FFChallenge_v5/background.csv', low_memory=False)
train = pd.read_csv('../data/FFChallenge_v5/train.csv', sep=',', header=0, index_col=0).dropna(how='all')
data = background.loc[train.index].join(train)

In [11]:
# convert all to numeric
le = LabelEncoder()
df = data.apply(le.fit_transform)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1466 entries, 6 to 4241
Columns: 13033 entries, challengeID to jobTraining
dtypes: int64(13033)
memory usage: 145.8 MB


In [12]:
df.head(5)

,challengeID,cf1intmon,cf1intyr,cf1lenhr,cf1lenmin,cf1twoc,cf1fint,cf1natsm,f1natwt,cf1natsmx,...,m5d20,m5k10,f5c6,k5f1,gpa,grit,materialHardship,eviction,layoff,jobTraining
challengeID,,,,,,,,,,,,,,,,,,,,,
6,0,0,0,3,42,0,1,0,0,0,...,0,0,82,150,13,9,1,0,0,0
7,1,0,0,2,42,0,1,0,0,0,...,0,0,188,224,6,8,0,0,0,0
9,2,0,0,3,42,0,1,0,0,0,...,41,0,188,91,5,11,2,0,0,0
10,3,0,0,3,27,0,1,0,0,0,...,0,0,188,16,9,8,1,0,2,0
13,4,0,0,3,37,0,1,0,0,0,...,0,0,113,2,7,11,2,0,0,1


# Export MI matrix

In [6]:
# discrete_features = df.select_dtypes(include=['object','int64']).columns

def get_miscore(df,targets):
    mi_data = []
    X = df.drop(targets,axis=1)
    for target in targets:
        # print(target)
        entry = {}
        Y = df[target]
        # check the use of discrete_features instead of label encoder
        mi_scores = mi_reg(X,Y)
        entry[target] = mi_scores
        mi_data.append(mi_scores)

    return pd.DataFrame(mi_data,index=targets,columns=X.columns)


In [7]:
# approx. run time: 5 min
miscores = get_miscore(df,targets=['gpa','grit','materialHardship','eviction','layoff','jobTraining']).T

In [8]:
miscores.to_csv('../metadata/miscores.csv')